# Bloodspot Processing

In [23]:
import json
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps
import biopsykit as bp
from biopsykit.io import biomarker

from empkins_io.datasets.d03.macro_ap01 import MacroBaseDataset

%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

[(0.0, 0.1843137254901961, 0.4235294117647059),
 (0.4666666666666667, 0.6235294117647059, 0.7098039215686275),
 (1.0, 0.7215686274509804, 0.10980392156862745),
 (0.0, 0.6392156862745098, 0.8784313725490196),
 (0.2627450980392157, 0.6901960784313725, 0.16470588235294117),
 (0.7843137254901961, 0.06274509803921569, 0.1803921568627451)]

In [25]:
deploy_type = "local"

In [26]:
config_dict = json.load(Path("../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
base_path

PosixPath('/Volumes/luca_ssd/Study_Data/2022_05_AP01_Macro')

In [27]:
path_out = base_path.joinpath("data_tabular/bloodspots/final")
bp.utils.file_handling.mkdirs([path_out])

In [28]:
dataset = MacroBaseDataset(base_path)
dataset

,subject,condition
0,VP_01,ftsst
1,VP_01,tsst
2,VP_02,ftsst
3,VP_02,tsst
4,VP_04,ftsst
...,...,...
73,VP_39,tsst
74,VP_40,ftsst
75,VP_40,tsst
76,VP_41,ftsst


## CRP

### Samples

In [29]:
crp_data = bp.io.biomarker.load_biomarker_results(
    base_path.joinpath("data_tabular/bloodspots/raw/crp_values.xlsx"), biomarker_type="crp", regex_str=r"(VP_\d+)-(T\w)-(B\w)"
)

crp_data = crp_data.join(dataset.day_condition_map).reset_index()
crp_data = crp_data.drop(columns="day").set_index(["subject", "condition", "sample"])
crp_data

crp
subject condition sample          
VP_01   tsst      B0       1.28780
                  B1       0.87412
        ftsst     B0       0.88948
                  B1       0.93484
VP_02   tsst      B0       0.61150
...                            ...
VP_40   tsst      B1      21.88960
VP_41   ftsst     B0       3.78896
                  B1       4.38420
        tsst      B0       5.11540
                  B1       3.43092

[164 rows x 1 columns]

In [30]:
# long format
crp_data = crp_data.pivot_table(index="subject", columns=["condition", "sample"], values="crp")

In [31]:
# squeeze multiindex
crp_data.columns = [f"{col[1]}_{col[0]}" for col in crp_data.columns]

In [33]:
crp_data = crp_data.add_prefix("crp_")

In [34]:
crp_data

,crp_B0_ftsst,crp_B1_ftsst,crp_B0_tsst,crp_B1_tsst
subject,,,,
VP_01,0.889480,0.934840,1.287800,0.874120
VP_02,0.429420,0.168260,0.611500,0.470740
VP_03,2.100160,1.232640,1.947040,1.549880
VP_04,6.011200,5.501600,6.434800,7.413400
VP_05,0.285412,0.549940,0.499500,0.221688
VP_06,1.365560,1.156440,0.951840,0.693660
VP_07,0.241160,0.149816,0.709680,0.189816
VP_08,4.369800,6.132400,NaN,0.247620
VP_09,1.911580,1.457760,1.133520,0.412560


In [35]:
crp_data.to_csv(path_out.joinpath("crp.csv"))